In [11]:
import os
os.environ['DJANGO_SETTINGS_MODULE'] = 'askcompnay.settings'
# django 3 부터 ASYNC 환경 세팅이 필요하다. 
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'true'

import django
django.setup()

In [12]:
# 바
from instagram.models import Post

In [14]:
# Cell 

qs = Post.objects.all()
print(type(qs))
print(qs.query)
qs

<class 'django.db.models.query.QuerySet'>
SELECT "instagram_post"."id", "instagram_post"."message", "instagram_post"."photo", "instagram_post"."is_public", "instagram_post"."created_at", "instagram_post"."updated_at" FROM "instagram_post"


<QuerySet [<Post: 첫 번쨰 메세지>, <Post: 2번째 메세지>, <Post: 세번쨰 메세지>, <Post: qwewq>]>

In [21]:
# 정렬은 넣어주는 것이 좋다. --> 모델에 default 정렬을 지정하는 방법도 있으며, 추후 진행 예정 
qs = Post.objects.all().order_by('-id')
print(qs.query)
qs

SELECT "instagram_post"."id", "instagram_post"."message", "instagram_post"."photo", "instagram_post"."is_public", "instagram_post"."created_at", "instagram_post"."updated_at" FROM "instagram_post" ORDER BY "instagram_post"."id" DESC


<QuerySet [<Post: qwewq>, <Post: 세번쨰 메세지>, <Post: 2번째 메세지>, <Post: 첫 번쨰 메세지>]>

In [22]:
qs = Post.objects.all().order_by('-id')[:2]
print(qs.query)
qs

SELECT "instagram_post"."id", "instagram_post"."message", "instagram_post"."photo", "instagram_post"."is_public", "instagram_post"."created_at", "instagram_post"."updated_at" FROM "instagram_post" ORDER BY "instagram_post"."id" DESC LIMIT 2


<QuerySet [<Post: qwewq>, <Post: 세번쨰 메세지>]>

In [23]:
for post in qs:
    print(post.id, post.message, post.created_at)

4 qwewq 2022-06-21 05:02:36.528183+00:00
3 세번쨰 메세지 2022-06-21 00:40:48.268101+00:00


In [26]:
for post in qs:
    print("id {id}, message: {message} ({created_at})".format(**post.__dict__))

id 4, message: qwewq (2022-06-21 05:02:36.528183+00:00)
id 3, message: 세번쨰 메세지 (2022-06-21 00:40:48.268101+00:00)


In [32]:
# filter에 OPTION은 [태그명]__[필터속성] 을 부여하여 작업을 하게 된다. 
qs = Post.objects.all().filter(message__icontains='첫번째')
print(qs.query)

SELECT "instagram_post"."id", "instagram_post"."message", "instagram_post"."photo", "instagram_post"."is_public", "instagram_post"."created_at", "instagram_post"."updated_at" FROM "instagram_post" WHERE "instagram_post"."message" LIKE %첫번째% ESCAPE '\'


In [34]:
# filter에 OPTION은 [태그명]__[필터속성] 을 부여하여 작업을 하게 된다. 
query = '메세지'
qs = Post.objects.all()
qs = qs.exclude(message__icontains=query)

print(qs.query)

SELECT "instagram_post"."id", "instagram_post"."message", "instagram_post"."photo", "instagram_post"."is_public", "instagram_post"."created_at", "instagram_post"."updated_at" FROM "instagram_post" WHERE NOT ("instagram_post"."message" LIKE %메세지% ESCAPE '\')


In [35]:
qs = Post.objects.all()
qs[1]

<Post: 2번째 메세지>

In [38]:
qs = Post.objects.all()
qs.get(id=2)

<Post: 2번째 메세지>

In [44]:
qs = Post.objects.all()
qs.get(id__lte=2) # less then , less than equal, grater than, grater than equal

MultipleObjectsReturned: get() returned more than one Post -- it returned 2!

In [46]:
qs.first()

<Post: 첫 번쨰 메세지>

In [47]:
qs.last()

<Post: qwewq>

In [50]:
print(qs.none().first())

None


In [51]:
from django.db.models import Q

In [54]:
qs = Post.objects.all()
qs = qs.filter(id__gte=2, message__icontains=query)
print(qs.query)

SELECT "instagram_post"."id", "instagram_post"."message", "instagram_post"."photo", "instagram_post"."is_public", "instagram_post"."created_at", "instagram_post"."updated_at" FROM "instagram_post" WHERE NOT ("instagram_post"."id" >= 2 AND "instagram_post"."message" LIKE %메세지% ESCAPE '\')


In [59]:
qs = Post.objects.all()
qs = qs.filter(Q(id__gte=2) & Q(message__icontains=query))
print(qs.query)

SELECT "instagram_post"."id", "instagram_post"."message", "instagram_post"."photo", "instagram_post"."is_public", "instagram_post"."created_at", "instagram_post"."updated_at" FROM "instagram_post" WHERE ("instagram_post"."id" >= 2 AND "instagram_post"."message" LIKE %메세지% ESCAPE '\')


In [60]:
qs = Post.objects.all()
qs = qs.filter(Q(id__gte=2) | Q(message__icontains=query))
print(qs.query)

SELECT "instagram_post"."id", "instagram_post"."message", "instagram_post"."photo", "instagram_post"."is_public", "instagram_post"."created_at", "instagram_post"."updated_at" FROM "instagram_post" WHERE ("instagram_post"."id" >= 2 OR "instagram_post"."message" LIKE %메세지% ESCAPE '\')


In [ ]:
cond = Q(id__get=2) | Q(message__icontains=query)
# cond&=